In [1]:
import pandas as pd
import os
import json
from tqdm import tqdm
pd.set_option('max_colwidth', 400)

In [2]:
import subprocess
import itertools

In [3]:
import pandas as pd
from scipy import spatial
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
from scipy import stats
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, LoggingHandler, util, models, evaluation, losses, InputExample

# Leitura da entrada

In [4]:
def transform_json(diario_json):
    dict_list =  []
    for key, value in json.loads(diario_json).items():
        for v in value:
            v["origin"] = key
            dict_list.append(v)
    return pd.DataFrame(dict_list)

In [5]:
dfs = []
directory = "../saida_segmentacao/AMM/"
for filename in tqdm(list(os.listdir(directory))):
    with open(directory + filename, "r") as jsonfile:
        diario_json = jsonfile.read()
        dfs.append(transform_json(diario_json))
diario_df = pd.concat(dfs)
diario_df.sample(3)

100%|████████████████████████████████████████████████████████████████████████████| 1640/1640 [00:10<00:00, 153.45it/s]


,titulo,subtitulo,materia,publicador,id,origin
217,GABINETE DO PREFEITO,PORTARIA N. 3072/2014,"Designa Membros da Comissão Municipal de Seleção para os Processos Seletivos Simplificados no Exercício de 2014 O Prefeito do Município de Rodeiro, Estado de Minas Gerais, no uso de atribuições que lhe confere o Art. 67, inciso VII, da Lei Orgânica Municipal, Resolve: Art. 1º Designar os membros da Comissão Municipal de Seleção dos Processos Seletivos Simplificados no Exercício de 2014 do Muni...",Frederico Pereira Paschoalino,1B8BA1EB,MUNICÍPIO DE RODEIRO
724,DEPARTAMENTO DE LICITAÇÃO,,"EXTRATO TERMO ADITIVO PL 019/2018, PP 009/2018 TERMO ADITIVO Nº 002 Contrato Administrativo nº 044/2018, PL nº 019/2018, PP nº 009/2018. Partes: Município de Senador Modestino Gonçalves-MG e a Empresa Gerir Administração e Serviços Ltda. ME, CNPJ nº 25.774.852/0001-67. Fica prorrogado o contrato original, passando a vigorar até o dia 31/12/2019. Objeto: CONTRATAÇÃO DE EMPRESA PARA PRESTAÇÃO DE...",Arnaldo de Oliveira Neto,8162A52A,PREFEITURA DE SENADOR MODESTINO GONÇALVES
418,DEPARTAMENTO DE COMPRAS E LICITAÇÕES,EXTRATO DA ATA DE REGISTRO DE PREÇOS Nº 183/2018,"ATA DE REGISTRO DE PREÇOS Nº 183/2018 PROCESSO Nº: 98/2018 MODALIDADE: Pregão nº 79/2018 CONTRATANTE: MUNICIPIO DE SANTO ANTÔNIO DO MONTE CONTRATADO: Sociedade Montense de Radiofusão Ltda OBJETO: Registro de preços para futuras e eventuais contratações de empresas especializadas na prestação de serviços de inserções radiofônicas e propaganda volante, atendendo a solicitação da Assessoria de Co...",Ana Luiza Morais Silva,32E9E6E3,PREFEITURA DE SANTO ANTÔNIO DO MONTE


In [6]:
diario_df["entidades"] = [{}] * len(diario_df)
diario_df["possiveis_consultas"] = [{}] * len(diario_df)

# Extraíndo entidades

In [4]:
os.chdir("../M05-dev/NER/M02/")

In [24]:
def get_combinations(lists):
    queries = []
    combinations = list(itertools.product(*lists))
    for comb in combinations:
        queries.append(" ".join(comb))
    return queries

def generate_queries(entities):
    queries = []
    types = {entity["label"]: [] for entity in entities}    
    for entity in entities:
        types[entity["label"]].append(entity["entity"])
    queries += get_combinations([types.get("PESSOA", []), types.get("ORGANIZACAO", [])])
    queries += get_combinations([types.get("PESSOA", []), types.get("LEGISLACAO", [])])
    queries += get_combinations([types.get("PESSOA", []), types.get("LOCAL", [])])
    queries += get_combinations([types.get("PESSOA", []), types.get("JURISPRUDENCIA", [])])
    queries += get_combinations([types.get("PESSOA", []), types.get("LOCAL", []), types.get("ORGANIZACAO", [])])

    queries = list(set(queries))
    return queries

In [9]:

os.environ["TOKENIZERS_PARALLELISM"] = "false"

SAMPLE_SIZE = 100000
diario_df_entidades = diario_df.sample(SAMPLE_SIZE).copy(deep=True).reset_index(drop=True)
for idx, row in tqdm(diario_df.iterrows(), total=SAMPLE_SIZE):
    try:
        text = row["materia"].replace("\n", "")
        output = subprocess.getstatusoutput(f'java -cp mp-ufmg-ner.jar:lib/* Pipeline -str "{text}"')
        output = eval(output[1][69:]) 
        diario_df_entidades.iloc[idx]["entidades"] = output["entities"]
        diario_df_entidades.iloc[idx]["possiveis_consultas"] = generate_queries(output["entities"])
    except Exception as e:
        print(e, idx)
        continue

  0%|▎                                                                        | 406/100000 [19:09<58:35:44,  2.12s/it]

invalid syntax (<string>, line 1) 84


  0%|▎                                                                        | 463/100000 [21:46<54:18:53,  1.96s/it]

invalid syntax (<string>, line 1) 141


  1%|▍                                                                        | 615/100000 [28:47<77:22:58,  2.80s/it]

...

invalid syntax (<string>, line 1) 259


 13%|████████▊                                                            | 12729/100000 [9:51:45<74:03:26,  3.05s/it]

invalid syntax (<string>, line 1) 39


 13%|████████▊                                                            | 12787/100000 [9:54:26<67:34:22,  2.79s/it]


KeyboardInterrupt: 

In [27]:
print(len(diario_df_entidades))
diario_df_entidades.head()

519


,titulo,subtitulo,materia,publicador,id,origin,entidades,possiveis_consultas
0,LICITAÇÕES E COMPRAS,EXTRATO DE LICITAÇÃO PREGÃO ELETRONICO 135/2017 Processo 0314/2017 - Pregão Eletrônico 0135/2017 – Objeto Aquisição de materiais de avaliação física para atender à Secretaria de Esporte e Lazer. Contratação EXCLUSIVAMENTE para Microempresa - ME ou Empresa de Pequeno Porte - EPP. Credenciamento: até as 12h do dia 23/10/2017. Apresentação das propostas: até as 13h do dia 23/10/2017. Início dos l...,,Adalberto da Silva Nogueira,75D060C0,PREFEITURA DE SÃO LOURENÇO,"[{'start': 0, 'end': 24, 'label': 'JURISPRUDENCIA', 'entity': 'PAL 005/2016 PP 003/2016'}, {'start': 242, 'end': 250, 'label': 'TEMPO', 'entity': '003/2016'}, {'start': 870, 'end': 874, 'label': 'TEMPO', 'entity': '2016'}, {'start': 913, 'end': 923, 'label': 'TEMPO', 'entity': '21/06/2016'}, {'start': 971, 'end': 985, 'label': 'LOCAL', 'entity': 'Rua Zeca Bruno'}, {'start': 1013, 'end': 1023, ...",[]
1,DEPARTAMENTO DE LICITAÇÕES,,"ABERTURA DO PREGÃO PRESENCIAL 58/16 Torna público a abertura do Pregão Presencial 58/16 no dia 07/11/16 às 09h30min, com credenciamento a partir das 09h. Objeto: contratação de empresa para execução de rede óptica subterrânea para interligação da escola municipal da Nilza Vieira à escola municipal de Lapinha e USF Lapinha com fornecimento de materiais. O edital na íntegra estará disponível na ...",André Luiz Fernandes,CA603261,PREFEITURA MUNICIPAL DE LAGOA SANTA,"[{'start': 50, 'end': 58, 'label': 'TEMPO', 'entity': '002/2016'}, {'start': 94, 'end': 102, 'label': 'TEMPO', 'entity': '002/2016'}, {'start': 161, 'end': 164, 'label': 'ORGANIZACAO', 'entity': 'CPL'}, {'start': 658, 'end': 690, 'label': 'ORGANIZACAO', 'entity': 'Associação Mineira de Municípios'}, {'start': 702, 'end': 718, 'label': 'PESSOA', 'entity': 'Diretoria Biênio'}, {'start': 719, 'en...","[ADRIANO JOSÉ SENADOR Associação Mineira de Municípios, ADRIANO JOSÉ SENADOR Caxambu Associação dos Municípios da Microrregião do Circuito, Teófilo Faustino Miranda Torres Duarte Minas Gerais CPL, Avenida Camilo Soares Caxambu, Geraldo Magela Barbosa 2º Tesoureiro Caxambu Associação Mineira de Municípios, ADRIANO JOSÉ SENADOR Associação dos Municípios da Microrregião do Circuito, Teófilo Faust..."
2,FUNDO MUNICIPAL DE SAÚDE DE CABECEIRA GRANDE,EXTRATO DA ATA DE REGISTRO DE PREÇO N.º 021/2020,"PROCESSO ADMINISTRATIVO LICITATÓRIO N.º 021/2020 PREGÃO PRESENCIAL – SRP N.º 020/2020 CONTRATADA: EQUIPAR MEDICO E HOSPITALAR LTDA – CNPJ: 25.725.813/0001-70 CONTRATANTE: Prefeitura Municipal De Cabeceira Grande CNPJ: 01.603.707/0001-55 Valor da Ata: R$ 49.944,00 (Quarenta e nove mil, novecentos e quarenta e quatro reais). Objeto: Formação de registro de preços para aquisição medicamentos rema...",Juliana Costa de Oliveira,9570AF41,PREFEITURA DE CABECEIRA GRANDE,"[{'start': 81, 'end': 99, 'label': 'TEMPO', 'entity': '30 DE MAIO DE 2016'}, {'start': 139, 'end': 156, 'label': 'ORGANIZACAO', 'entity': 'Consórcio Público'}, {'start': 203, 'end': 248, 'label': 'ORGANIZACAO', 'entity': 'Emergência da Macrorregião do Triângulo Norte'}, {'start': 278, 'end': 282, 'label': 'TEMPO', 'entity': '2015'}, {'start': 300, 'end': 353, 'label': 'LEGISLACAO', 'entity': '...","[LORENA CÂNDIDA VIEIRA BORGES CRC Emergência da Macrorregião do Triângulo Norte, LORENA CÂNDIDA VIEIRA BORGES CRC Portaria n. 72/2012 da Secretaria do Tesouro Nacional, GILMAR ALVES MACHADO Consórcio Público, GILMAR ALVES MACHADO Portaria n. 72/2012 da Secretaria do Tesouro Nacional, LORENA CÂNDIDA VIEIRA BORGES CRC Tesouro Nacional, GILMAR ALVES MACHADO art.s 17 e 28 do Contrato do Consórcio ..."
3,LICITAÇÕES E COMPRAS,"EXTRATO DE LICITAÇÃO PREGÃO ELETRONICO 038/2018 Processo 0104/2018 - Pregão Eletrônico 0038/2018 – Objeto Contratação de empresa especializada para planejamento, desenvolvimento, organização, contratação, supervisão, coordenação e execução dos trabalhos do evento ""VI Copa de Marcha do Cavalo Mangalarga Marchador, Exposição de Mini-An

# Treinamento

In [11]:
diario_df_entidades = diario_df_entidades[diario_df_entidades["entidades"].apply(bool)]

In [5]:
train_df = pd.read_csv("temp.csv")

In [28]:
train_df = diario_df_entidades[["materia", "possiveis_consultas"]].explode("possiveis_consultas")
train_df.columns = ["doc", "query"]
train_df.dropna()

,doc,query
1,"ABERTURA DO PREGÃO PRESENCIAL 58/16 Torna público a abertura do Pregão Presencial 58/16 no dia 07/11/16 às 09h30min, com credenciamento a partir das 09h. Objeto: contratação de empresa para execução de rede óptica subterrânea para interligação da escola municipal da Nilza Vieira à escola municipal de Lapinha e USF Lapinha com fornecimento de materiais. O edital na íntegra estará disponível na ...",ADRIANO JOSÉ SENADOR Associação Mineira de Municípios
1,"ABERTURA DO PREGÃO PRESENCIAL 58/16 Torna público a abertura do Pregão Presencial 58/16 no dia 07/11/16 às 09h30min, com credenciamento a partir das 09h. Objeto: contratação de empresa para execução de rede óptica subterrânea para interligação da escola municipal da Nilza Vieira à escola municipal de Lapinha e USF Lapinha com fornecimento de materiais. O edital na íntegra estará disponível na ...",ADRIANO JOSÉ SENADOR Caxambu Associação dos Municípios da Microrregião do Circuito
1,"ABERTURA DO PREGÃO PRESENCIAL 58/16 Torna público a abertura do Pregão Presencial 58/16 no dia 07/11/16 às 09h30min, com credenciamento a partir das 09h. Objeto: contratação de empresa para execução de rede óptica subterrânea para interligação da escola municipal da Nilza Vieira à escola municipal de Lapinha e USF Lapinha com fornecimento de materiais. O edital na íntegra estará disponível na ...",Teófilo Faustino Miranda Torres Duarte Minas Gerais CPL
1,"ABERTURA DO PREGÃO PRESENCIAL 58/16 Torna público a abertura do Pregão Presencial 58/16 no dia 07/11/16 às 09h30min, com credenciamento a partir das 09h. Objeto: contratação de empresa para execução de rede óptica subterrânea para interligação da escola municipal da Nilza Vieira à escola municipal de Lapinha e USF Lapinha com fornecimento de materiais. O edital na íntegra estará disponível na ...",Avenida Camilo Soares Caxambu
1,"ABERTURA DO PREGÃO PRESENCIAL 58/16 Torna público a abertura do Pregão Presencial 58/16 no dia 07/11/16 às 09h30min, com credenciamento a partir das 09h. Objeto: contratação de empresa para execução de rede óptica subterrânea para interligação da escola municipal da Nilza Vieira à escola municipal de Lapinha e USF Lapinha com fornecimento de materiais. O edital na íntegra estará disponível na ...",Geraldo Magela Barbosa 2º Tesoureiro Caxambu Associação Mineira de Municípios
...,...,...
519,"PROCESSO DE LICITAÇÃO Nº 058/2.017 PREGÃO PRESENCIAL Nº 034/2.017 REGISTRO DE PREÇO Nº 028/2017 A Prefeitura Municipal de São Francisco do Glória torna público nos termos da Lei Federal nº 10.520/02 C/C Lei Federal nº 8.666/93 e a Pregoeira, tornam público que fará realizar o Processo de Licitação nº. 058/2017/Pregão Presencial nº. 034/2017/Registro de Preço nº. 028/2.017, cujo objeto é o regi...",BRENO FERREIRA RIOS CNPJ 55.956.510/0001
519,"PROCESSO DE LICITAÇÃO Nº 058/2.017 PREGÃO PRESENCIAL Nº 034/2.017 REGISTRO DE PREÇO Nº 028/2017 A Prefeitura Municipal de São Francisco do Glória torna público nos termos da Lei Federal nº 10.520/02 C/C Lei Federal nº 8.666/93 e a Pregoeira, tornam público que fará realizar o Processo de Licitação nº. 058/2017/Pregão Presencial nº. 034/2017/Registro de Preço nº. 028/2.017, cujo objeto é o regi...",PABLO LUIZ LOPES BRASIL SAAE
519,"PROCESSO DE LICITAÇÃO Nº 058/2.017 PREGÃO PRESENCIAL Nº 034/2.017 REGISTRO DE PREÇO Nº 028/2017 A Prefeitura Municipal de São Francisco do Glória torna público nos termos da Lei Federal nº 10.520/02 C/C Lei Federal nº 8.666/93 e a Pregoeira, tornam público que fará realizar o Processo de Licitação nº. 058/2017/Pregão Presencial nº. 034/2017/Registro de Preço nº. 028/2.017, cujo objeto é o regi...",COMERCIO LTDA SAAE
519,"PROCESSO DE LICITAÇÃO Nº 058/2.017 PREGÃO PRESENCIAL Nº 034/2.017 REGISTRO DE PREÇO Nº 028/2017 A Prefeitura Municipal de São Francisco do Glória torna público nos termos da Lei Federal nº 10.520/02 C/C Lei Federal nº 8.666/93 e a Pregoeira, tornam público que fará realizar o Processo de Licitação nº. 058/2017/Pregão Presencial nº. 034/2017/R

In [40]:
train_df["len"] = train_df["query"].apply(len)
train_df = train_df[train_df["len"] > 10]
train_df = train_df[train_df["len"] < 100]

In [42]:
train_df["doc"] = train_df["doc"].apply(str)
train_df["query"] = train_df["query"].apply(str)

In [72]:
def average_similarity():
    false_pairs, true_pairs = 0, 0
    SAMPLE_SIZE = 1000
    df_sample = train_df.sample(SAMPLE_SIZE, random_state=42)
    for i in tqdm(range(1, SAMPLE_SIZE)):
        row, neg = df_sample.iloc[i], df_sample.iloc[i - 1]
        output = model.encode([row["query"], row["doc"], neg["doc"]]), 

        true_pairs += 1 - spatial.distance.cosine(output[0][0], output[0][1])
        false_pairs += 1 - spatial.distance.cosine(output[0][0], output[0][2])

    print("Média de similaridade dos positivos: ", true_pairs / SAMPLE_SIZE)
    print("Média de similaridade dos negativos: ", false_pairs / SAMPLE_SIZE)

In [73]:
from sentence_transformers import SentenceTransformer, models
from torch import nn

word_embedding_model = models.Transformer('neuralmind/bert-base-portuguese-cased', max_seq_length=250)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [74]:
average_similarity()

100%|███████████████████████████████████████████████████████████████████████████████| 999/999 [00:24<00:00, 40.59it/s]

Média de similaridade dos positivos:  0.608215974509716
Média de similaridade dos negativos:  0.6101312281638384


In [75]:
examples = []
SAMPLE_SIZE = 3000
for i in tqdm(range(1, len(train_df.sample(SAMPLE_SIZE, random_state=24))), position=0, leave=True, total=SAMPLE_SIZE):
  row, neg = train_df.iloc[i], train_df.iloc[i - 1]
  examples.append(InputExample(texts=[row["query"], row["doc"], neg["doc"]]))

100%|██████████████████████████████████████████████████████████████████████████▉| 2999/3000 [00:00<00:00, 4335.34it/s]


In [76]:
dataloader = DataLoader(examples, shuffle=True, batch_size=8)
train_loss = losses.TripletLoss(model)
model.fit(train_objectives=[(dataloader, train_loss)], epochs=1, warmup_steps=50, use_amp=True)

In [77]:
average_similarity()

100%|███████████████████████████████████████████████████████████████████████████████| 999/999 [00:25<00:00, 39.32it/s]

Média de similaridade dos positivos:  0.5421588616222143
Média de similaridade dos negativos:  0.5392619294971228


# Possíveis consultas

In [71]:
train_df["query"].to_frame()

,query
1,ADRIANO JOSÉ SENADOR Associação Mineira de Municípios
2,ADRIANO JOSÉ SENADOR Caxambu Associação dos Municípios da Microrregião do Circuito
3,Teófilo Faustino Miranda Torres Duarte Minas Gerais CPL
4,Avenida Camilo Soares Caxambu
5,Geraldo Magela Barbosa 2º Tesoureiro Caxambu Associação Mineira de Municípios
...,...
5491,BRENO FERREIRA RIOS CNPJ 55.956.510/0001
5492,PABLO LUIZ LOPES BRASIL SAAE
5493,COMERCIO LTDA SAAE
5494,ANALITICA QUIMICA LIO SERUM PROD LABORATORIAIS E HOSPITALARES LTDA
